
# STAT41130 — wandb + PyTorch (No-Prompt Login)

This notebook shows a tiny PyTorch model **with Weights & Biases tracking** — and it **won’t prompt** for an API key in VS Code notebooks.

**Login flow used here**
1. If `WANDB_API_KEY` env var is set → log in with it (online).
2. Else if you paste an **inline key** into the cell below → log in with it (online).
3. Otherwise → **auto-switch to offline mode** (no prompt), logs saved locally to sync later.

**Quick start**
- Online (recommended): set env vars in a `.env` file or your shell:
  ```bash
  export WANDB_API_KEY=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
  export WANDB_ENTITY=your-team-or-user
  export WANDB_PROJECT=stat41130-demo
  ```
- Offline: do nothing; the cell below will set `WANDB_MODE=offline` automatically.
- Optional one-time terminal login also works: `wandb login`



## 0) Install W&B (one-time)
If missing, install with pip.


In [ ]:

# %pip install --quiet wandb


In [1]:

import os, math, time
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

try:
    import wandb
except ImportError as e:
    raise SystemExit("wandb is not installed. Run `pip install wandb` and re-run this cell.") from e



## 1) Configure W&B (no interactive prompt)
- **Preferred:** set `WANDB_API_KEY`, `WANDB_ENTITY`, `WANDB_PROJECT` in your environment.
- **Inline key (temporary):** paste your key into `INLINE_KEY` **only on your machine** (don’t commit to Git).
- If neither is provided, we **default to offline mode** to avoid prompts.


In [2]:

CFG = {
    "seed": 42,
    "epochs": 40,
    "batch_size": 64,
    "learning_rate": 0.05,
    "hidden": 16,
    "dataset": "synthetic-rings",
    "architecture": "MLP(2->16->1)",
}

# Inline key (OPTIONAL). Leave empty to avoid hardcoding secrets.
INLINE_KEY = ""  # e.g., "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

# Pick up env vars; you can set these in a .env file or your shell.
ENTITY  = os.getenv("WANDB_ENTITY", "your-wandb-entity")   # change if desired
PROJECT = os.getenv("WANDB_PROJECT", "stat41130-demo")     # change if desired

# Decide login method without prompting.
api_key = os.getenv("WANDB_API_KEY") or INLINE_KEY
if api_key:
    try:
        wandb.login(key=api_key)
        print("[wandb] Logged in with provided API key.")
    except Exception as e:
        print(f"[wandb] Login failed: {e} — switching to offline mode.")
        os.environ["WANDB_MODE"] = "offline"
else:
    # No key provided → offline by default to avoid interactive prompt in VS Code
    if os.getenv("WANDB_MODE", "").lower() != "offline":
        os.environ["WANDB_MODE"] = "offline"
    print("[wandb] No API key found — using OFFLINE mode (set WANDB_API_KEY to enable online logging).")

run = wandb.init(
    entity=ENTITY,
    project=PROJECT,
    config=CFG,
    name=f"mlp-noprompt-{int(time.time())}",
    notes="STAT41130 minimal PyTorch + wandb (no prompt)",
)
print("W&B run:", (run.url if getattr(run, "url", None) else "offline/no-url"))


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\andre\_netrc
wandb: Currently logged in as: andrew-parnell (andrew-parnell-university-college-dublin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[wandb] Logged in with provided API key.


W&B run: https://wandb.ai/andrew-parnell-university-college-dublin/stat41130-demo/runs/ocxhs34b



## 2) Make a tiny synthetic dataset
2D inputs, two classes with slightly different radii (quick to train).


In [3]:

def make_tiny_dataset(n_train=1000, n_val=300, seed=0):
    g = torch.Generator().manual_seed(seed)
    def sample(n):
        theta = torch.rand(n, generator=g) * 2*math.pi
        y = (torch.rand(n, generator=g) > 0.5).float()
        r = (1.0 + y*0.6) + 0.15*torch.randn(n, generator=g)
        x1 = r * torch.cos(theta)
        x2 = r * torch.sin(theta)
        X = torch.stack([x1, x2], dim=1)
        return X, y.unsqueeze(1)
    Xtr, ytr = sample(n_train)
    Xva, yva = sample(n_val)
    return (Xtr, ytr), (Xva, yva)

(Xtr, ytr), (Xva, yva) = make_tiny_dataset(seed=CFG["seed"])
train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=CFG["batch_size"], shuffle=True)
val_loader   = DataLoader(TensorDataset(Xva, yva), batch_size=CFG["batch_size"], shuffle=False)

Xtr.shape, ytr.shape, Xva.shape, yva.shape


(torch.Size([1000, 2]),
 torch.Size([1000, 1]),
 torch.Size([300, 2]),
 torch.Size([300, 1]))


## 3) Define a tiny MLP classifier
One hidden layer with ReLU; **BCEWithLogitsLoss** for binary classification.


In [4]:

class TinyMLP(nn.Module):
    def __init__(self, hidden=16):
        super().__init__()
        self.fc1 = nn.Linear(2, hidden)
        self.fc2 = nn.Linear(hidden, 1)
    def forward(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h)  # logits

model = TinyMLP(hidden=CFG["hidden"])
opt = torch.optim.Adam(model.parameters(), lr=CFG["learning_rate"])
criterion = nn.BCEWithLogitsLoss()

# Track gradients/parameters in W&B (harmless in offline too)
wandb.watch(model, criterion=criterion, log="all", log_freq=10)
model


TinyMLP(
  (fc1): Linear(in_features=2, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


## 4) Train and log metrics
Logs: **loss**, **accuracy**, and a simple **decision boundary plot** (every 10 epochs).


In [5]:

def accuracy_from_logits(logits, y):
    probs = torch.sigmoid(logits)
    preds = (probs >= 0.5).float()
    return (preds.eq(y).float().mean().item())

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0
    for xb, yb in loader:
        logits = model(xb)
        loss = criterion(logits, yb)
        acc = accuracy_from_logits(logits, yb)
        bsz = xb.size(0)
        total_loss += loss.item() * bsz
        total_acc  += acc * bsz
        n += bsz
    return total_loss / n, total_acc / n

def plot_decision_boundary(model, X, y, step=0.05):
    # decision surface
    x_min, x_max = X[:,0].min().item()-0.5, X[:,0].max().item()+0.5
    y_min, y_max = X[:,1].min().item()-0.5, X[:,1].max().item()+0.5
    xs = torch.arange(x_min, x_max, step)
    ys = torch.arange(y_min, y_max, step)
    xx, yy = torch.meshgrid(xs, ys, indexing='ij')
    grid = torch.stack([xx.reshape(-1), yy.reshape(-1)], dim=1)
    with torch.no_grad():
        zz = torch.sigmoid(model(grid)).reshape_as(xx)
    fig, ax = plt.subplots(figsize=(4,4))
    ax.contourf(xx, yy, zz, levels=20, alpha=0.6)  # no explicit colors
    # overlay points without specifying colors; use different markers
    y_flat = y.squeeze()
    mask0 = (y_flat == 0)
    mask1 = ~mask0
    ax.scatter(X[mask0,0], X[mask0,1], s=10, marker='o', edgecolor='k', label='class 0')
    ax.scatter(X[mask1,0], X[mask1,1], s=10, marker='x', label='class 1')
    ax.legend(loc="best")
    ax.set_title("Decision boundary (prob class=1)")
    ax.set_xlabel("x1"); ax.set_ylabel("x2")
    fig.tight_layout()
    return fig

best_val_acc = 0.0
for epoch in range(1, CFG["epochs"] + 1):
    model.train()
    for xb, yb in train_loader:
        logits = model(xb)
        loss = criterion(logits, yb)
        opt.zero_grad()
        loss.backward()
        opt.step()

    tr_loss, tr_acc = evaluate(model, train_loader)
    va_loss, va_acc = evaluate(model, val_loader)

    log_dict = {"epoch": epoch, "train/loss": tr_loss, "train/acc": tr_acc,
                "val/loss": va_loss, "val/acc": va_acc}
    if epoch % 10 == 0 or epoch == 1:
        fig = plot_decision_boundary(model, Xtr, ytr)
        log_dict["plots/decision_boundary"] = wandb.Image(fig)
        plt.close(fig)

    wandb.log(log_dict)

    if va_acc > best_val_acc:
        best_val_acc = va_acc
        path = "best_model.pt"
        torch.save({"state_dict": model.state_dict(), "cfg": CFG}, path)
        artifact = wandb.Artifact("tiny_mlp_best", type="model")
        artifact.add_file(path)
        wandb.log_artifact(artifact)

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | train_acc={tr_acc:.3f} val_acc={va_acc:.3f}")

print("Training complete. Best val acc:", best_val_acc)


Epoch 001 | train_acc=0.712 val_acc=0.673
Epoch 010 | train_acc=0.973 val_acc=0.983
Epoch 020 | train_acc=0.973 val_acc=0.980
Epoch 030 | train_acc=0.976 val_acc=0.983
Epoch 040 | train_acc=0.972 val_acc=0.980
Training complete. Best val acc: 0.9833333365122477


In [6]:

# Finish the W&B run (uploads if online; stores locally if offline).
wandb.finish()


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/acc,▁▆▇███▇███▇██████▇▇████████████▇█▇██████
train/loss,█▆▄▃▂▂▂▂▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁
val/acc,▁▅▇▇▇█▇███▇███████▇████████████▇████████
val/loss,█▆▄▃▂▂▂▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁
epoch,40
train/acc,0.972
train/loss,0.07444
val/acc,0.98
val/loss,0.05882



## 5) Syncing offline runs later
If you ran in offline mode, W&B saved runs locally (e.g., `wandb/offline-run-*`). To sync later from a terminal:
```bash
wandb login  # if you haven't already
wandb sync wandb/offline-run-*
```
